# Migration between Scotland and overseas by age

Tabs: `AG 2001-02`, etc.

In [1]:
from gssutils import *

if is_interactive():
    scraper = Scraper('https://www.nrscotland.gov.uk/statistics-and-data/statistics/' \
                      'statistics-by-theme/migration/migration-statistics/migration-between-scotland-and-overseas')
#     scraper.run()
    distribution = scraper.distribution(
        mediaType='application/vnd.ms-excel',
        title='Migration between Scotland and overseas by age')
    display(distribution)
    tabs = [tab for tab in distribution.as_databaker() if tab.name.startswith('AG ')]
    display([tab.name for tab in tabs])
    

['AG 2001-02',
 'AG 2002-03',
 'AG 2003-04',
 'AG 2004-05',
 'AG 2005-06',
 'AG 2006-07',
 'AG 2007-08',
 'AG 2008-09',
 'AG 2009-10',
 'AG 2010-11',
 'AG 2011-12',
 'AG 2012-13',
 'AG 2013-14',
 'AG 2014-15',
 'AG 2015-16',
 'AG 2016-17']

In [2]:
Final_table = pd.DataFrame()

In [3]:
for tab in [t for t in tabs if t.name.startswith('AG ')]:
    observations = tab.excel_ref("B7").expand(RIGHT).expand(DOWN).is_not_blank().is_not_whitespace()
    area = tab.excel_ref('A').expand(DOWN).by_index([4,11,18])
    age = tab.excel_ref('B5').expand(RIGHT).is_not_blank()
    flow = tab.excel_ref('A7').expand(DOWN)
    per = tab.excel_ref('B6').expand(RIGHT).is_not_blank()
    Dimensions = [
            HDimConst('Mid Year',str(tab.name[3:7])+'-06-30T00:00:00/P1Y'),
            HDim(flow,'Flow',DIRECTLY, LEFT),
            HDim(age,'Age',DIRECTLY, ABOVE),
            HDim(per,'per',DIRECTLY, ABOVE),
            HDimConst('Measure Type', 'Count'),
            HDimConst('Unit','People'),
            HDimConst('Sex','T'),
            HDim(area,'Foreign geography',CLOSEST,ABOVE)
            ]
    c1 = ConversionSegment(observations, Dimensions, processTIMEUNIT=True)
    new_table = c1.topandas()
    new_table = new_table[new_table['per'] != '%']
    new_table['Value'] = pd.to_numeric(new_table['OBS'], errors='coerce').fillna(0)
    new_table['Value'] = new_table['Value'].astype(int)
    new_table = new_table[new_table['Flow'] != '']
    new_table['Flow'] = new_table['Flow'].map(
    lambda x: {
        'IN' : 'Inflow', 
        'OUT' : 'Outflow',
        'NET': 'Balance' 
        }.get(x, x))
    new_table['Age'] = new_table['Age'].map(lambda cell:cell.replace('+', '-plus'))
    new_table['Age'] = new_table['Age'].map(lambda cell:cell.replace('All ages', 'all'))
    new_table['Age'] = 'nrs/' + new_table['Age']
    new_table = new_table[new_table['Value'] != 0 ]
    new_table['Mid Year'] = new_table['Mid Year'].map(lambda x: str(x)[0:4]) + '-06-30T00:00:00/P1Y'
    new_table['Domestic geography'] = 'Scotland'
    new_table = new_table[['Domestic geography','Foreign geography','Mid Year','Sex','Age','Flow','Measure Type','Value','Unit']]
    Final_table = pd.concat([Final_table, new_table])

In [6]:
Final_table['Foreign geography'] = Final_table['Foreign geography'].map(
    lambda x: {
        'Movements between Scotland and the rest of the UK1' : 'nrs/rest-of-the-uk', 
        'Movements between Scotland and overseas (including asylum seekers; excluding unmeasured migration adjustment)2, 3' : 'nrs/overseas',
        'Total net migration (including asylum seekers)3' : 'nrs/total',
        'Movements between Scotland and overseas (including asylum seekers; excluding unmeasured migration adjustment)2,3': 'nrs/overseas',
        'Movements between Scotland and Overseas (including asylum seekers)2':'nrs/overseas',
        'Total net migration (including asylum seekers)' : 'nrs/total'
        }.get(x, x))